# Demonstration of a Quantum Variational Classification algorithm using the PT Series

This notebook demonstrates a simple quantum classification algorithm using the PT Series, where we determine whether some random points on a plane are located within a circle. The input points are encoded into beam splitter angles using a linear function, and the output states are transformed into class probabilities using a neural network with one hidden layer. This is an example of a hybrid quantum/classical neural network.

<center><img src="./figures/classifier_model.png" alt="Illustration of a hybrid neural network" width="400"/></center>
<center>Figure 1: Illustration of a hybrid quantum/classical neural network, where the PT Series acts as a quantum layer between two classical neural networks.</center>

In [ ]:
# First, perform the relevant imports and navigate to the root folder
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

if os.getcwd().endswith("notebooks"):
    os.chdir("..")

from ptseries.models.utils import calculate_n_params, calculate_output_dim
from ptseries.models.pt_layer import PTLayer
from ptseries.algorithms.classifiers import VariationalClassifier
from ptseries.algorithms.classifiers.utils import create_dataloader

# Creating a simple dataset
The dataset consists of 200 randomly selected points in the 2D box [-2, 2], where points that are within a radius of sqrt(2) from the center have yellow labels and the other points have blue labels.

In [ ]:
data = 4 * torch.rand((200, 2)) - 2
labels = (
    torch.where(data[:, 0] ** 2 + data[:, 1] ** 2 < 2, 1, 0)
    .unsqueeze(1)
    .to(torch.float32)
)
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(data[:, 0], data[:, 1], c=labels)
ax.set_aspect("equal", "box")
plt.show()

In PyTorch, it is common to work with dataloaders that simplify the training by iterating over a dataset in a very efficient way.

In [ ]:
train_dataloader = create_dataloader(data, labels, batch_size=16)

# Defining the model

The following syntax is the one usually used in PyTorch to define a classical model: we define blocs to which the model input x is sent during the forward pass. The only difference with a classical PyTorch model is the use of the object PTLayer, which describes the PT Series quantum device. Here the PTLayer has 2 input features corresponding to the 2 variable parameters in a 3-mode and 1-loop PT Series, and 3 outputs corresponding to the average number of photons in the 3 output modes.

In [ ]:
class Model(nn.Module):
    def __init__(self, tbi_params=None):
        super().__init__()

        self.input_size = 2
        self.input_state = (0, 1, 0)
        self.tbi_params = tbi_params
        self.observable = "avg-photons"

        # we use calculate_n_params to determine the number of beam splitter angles
        n_params = calculate_n_params(self.input_state, tbi_params=self.tbi_params)
        n_outputs = calculate_output_dim(
            self.input_state, tbi_params=self.tbi_params, observable=self.observable
        )

        self.net = nn.Sequential(
            nn.Linear(self.input_size, 100),
            nn.ReLU(),
            nn.Linear(100, n_params),
            PTLayer(
                self.input_state,
                in_features=n_params,
                observable=self.observable,
                tbi_params=self.tbi_params,
                n_samples=200,
            ),
            nn.Linear(n_outputs, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
model = Model()
print(model)

We can also display all the parameters of the PT Series by using the `print_info` method of the PTLayer object.

In [ ]:
model.net[3].print_info()

# Training our model
To train our model, we can directly use ORCA's SDK tools. The object `VariationalClassifier` in `ptseries.algorithms` can train any model (classical and/or quantum).

In [ ]:
loss_function = nn.BCELoss()
classifier = VariationalClassifier(model, loss_function)

In [ ]:
# Start the training loop. This takes a few seconds
classifier.train(
    train_dataloader,  # PyTorch dataloader containing the training data
    learning_rate=1e-2,
    epochs=5,
    print_frequency=5,
    verbose=True,
)

# Model evaluation
We now evaluate the learned model on some randomly selected test data, and observe that the model has generally correctly learned to separate the two classes. This is a simple demonstration of a quantum variational classifier using a single photon and simple encoding and decoding schemes; better performance can be achieved with more complex schemes and longer training runs.

In [ ]:
# Create some new test data and perform inference
data_test = 4 * torch.rand((200, 2)) - 2
classifier.model.eval()  # this sets PTLayer in eval mode
predictions = classifier.forward(data_test)

# Convert predictions to blue/yellow binary labels and plot the result
binarized_predictions = torch.where(predictions < 0.5, 0, 1).unsqueeze(1)
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(data_test[:, 0], data_test[:, 1], c=binarized_predictions)
ax.set_aspect("equal", "box")
plt.show()